In [20]:
import pandas as pd
import numpy as np


num_cities = 1000

np.random.seed(107)

cities = []
for i in range(num_cities):
    city = {
        'Population': np.random.randint(50000, 10000000),
        'Area': np.random.randint(50, 1000),
        'GDP': np.random.uniform(5, 100),
        'High education': np.random.uniform(0, 1),
        'Ad budget': np.random.uniform(0, 10000)
    }
    
    
    city['Income'] = np.random.normal(1000, 10000)
    cities.append(city)


df = pd.DataFrame(cities)
df.head()


In [22]:
df.describe()


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X = df[['Population', 'Area', 'GDP', 'High education', 'Ad budget']].values
y = df['Income'].values

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  #
    Dense(64, activation='relu'),  
    Dense(1)  
])


model.compile(optimizer='adam', loss='mean_squared_error')


model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


In [4]:
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")


In [5]:
new_city = np.array([[800000, 80, 0.7, 0.9, 50000]])
new_city_scaled = scaler.transform(new_city)
predicted_income = model.predict(new_city_scaled)
print(f"Predicted Income: {predicted_income[0]}")
